# Advent of Code 2018, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202018%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [1]:
⍝ Helper functions and common settings
⎕IO←0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
ilines←{⍎¨lines ⍵}
line←{⊃lines ⍵}
sorted←{⍵[⍋⍵]}
sortbycol←⊢⌷⍨∘⊂∘⍒⌷⍤1
pairs←{↓(2÷⍨≢⍵) 2⍴⍵}
bin←{(32⍴2)⊤⍵}
dec←2∘⊥
and←{(bin ⍺)∧bin ⍵}
or←{(bin ⍺)∨bin ⍵}
rs←⊢∘-∘≢↑↓⍨∘-⍨

done

Rebuilding user command cache... done

In [2]:
⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Chronal Calibration
https://adventofcode.com/2018/day/1

In [39]:
DAY1←ilines'data/2018/01.txt'

In [87]:
⊢Part1←+/DAY1
assert 477=Part1

477

For part 2 we're looking for the first frequency to recur. If we get to the end, take the last value and use that as the start value for the next iteration. We look for the first duplicate using Dyalog's [unique mask](https://aplwiki.com/wiki/Nub_Sieve) function (monadic `≠`), introduced in v18.

In [32]:
Day1p2←{t←⍬⋄0{t,←1↓+\⍺,⍵⋄1=≢e←t[⍸<\~≠t]:⊃e⋄⍵∇⍨⊃¯1↑t}⍵}

In [86]:
⊢Part2←Day1p2 DAY1
assert 390=Part2

390

### Day 2: Inventory Management System
https://adventofcode.com/2018/day/2

In [85]:
⊢Part1←×/+/⍉↑2 3∘∊¨(≢⊢)⌸¨DAY2←lines'data/2018/02.txt'
assert 8118=Part1

8118

In [84]:
⊢Part2←(=⌿l)/0⌷l←↑DAY2[⍸{1∊⍵}¨↓DAY2∘.{+/≠/⍺,⍪⍵}DAY2]
assert 'jbbenqtlaxhivmwyscjukztdp'≡Part2

┌→────────────────────────┐
│jbbenqtlaxhivmwyscjukztdp│
└─────────────────────────┘

### Day 3: No Matter How You Slice It
https://adventofcode.com/2018/day/3

In [128]:
DAY3←⍎¨¨'\d+'⎕s'&'¨lines'data/2018/03.txt'

In [152]:
R←1000 1000⍴0⋄_←{R[⍵]+←1}¨CL←{,((2⊃⍵)+⍳4⊃⍵)∘.,((1⊃⍵)+⍳3⌷⍵)}¨DAY3 ⍝ Convert origin + dx dy to coord pairs

In [153]:
⊢Part1←+/2≤∊R
assert 104126=Part1

104126

In [154]:
⊢Part2←1+⊃⍸{∧/1=R[⍵]}¨CL ⍝ Claim numbers are 1-indexed, so need to add 1
assert 695=Part2          

695

### Day 4: Repose Record
https://adventofcode.com/2018/day/4

Basically a series of set operations -- sort based on date, then split and merge into chunks based on each guard's log lines. Pair up into sleep-wake pairs and expand into the corresponding minute range. We can then use key `⌸` to find the most frequent minute for the guard with the most sleep minutes.

The regex

   '(?|.*(#\d+).*|.*:(\d\d)).*'
   
is a _branch reset_ `(?| ...|... )`, which means that the result of whichever of the two branches that matches can be referred to as `\1` in the replacement.

In [118]:
'iotag'⎕CY'dfns'
DAY4←sorted lines'data/2018/04.txt'

In [119]:
parsed←'(?|.*(#\d+).*|.*:(\d\d)).*'⎕r'\1'⊢DAY4 ⍝ Pick out guard number, and minutes for each sleep/wake item

In [120]:
guards←{0⊃0⊃⍵}¨merged←(≠{⍎1↓0⊃⍵}¨grouped)⊂grouped←sorted parsed⊂⍨'#'∘(1∊⍷)¨parsed

In [121]:
sleepiest←⊃⍒≢¨sleep←{∊{(⍎⍺) iotag ¯1+⍎⍵}/↑⍵}¨pairs¨{⊃,/{1↓⍵}¨⍵}¨merged

In [122]:
⊢Part1←(⍎1↓sleepiest⊃guards)×⊃((⊣/⊢⍤/⍨∘(⌈/=⊢)⊢/){⍺(≢⍵)}⌸)sleepiest⊃sleep
assert 138280=Part1

138280

For part 2, we seek the guard which has the highest minute sleep frequency. We have most of the parts already.

In [412]:
f←⊃⍒{⌈/1⌷⍉⍵}¨freq←({⍺,(≢⍵)}⌸)¨sleep

In [417]:
⊢Part2←(⍎1↓f⊃guards)×⊃(⊣/⊢⍤/⍨∘(⌈/=⊢)⊢/)f⊃freq
assert 89347=Part2

89347

### Day 5: Alchemical Reduction
https://adventofcode.com/2018/day/5

In [ ]:
DAY5←line'data/2018/05.txt'

In [418]:
r←∊'|'(1↓∘,,⍤0)p,⌽¨p←(⎕C⎕A),¨⎕A ⍝ Make the regex aA|Aa|bB|Bb ... etc

In [433]:
⊢Part1←≢{r⎕r''⊢⍵}⍣≡⊢DAY5        ⍝ Apply regex replace until no change
assert 9172=Part1

9172

In [435]:
⊢Part2←⌊/{≢{r⎕r''⊢⍵}⍣≡⊢(⍵⎕r''⍠'IC'1)DAY5}¨⎕A ⍝ This several minutes to run
assert 6550=Part2

6550

### Day 6: Chronal Coordinates
https://adventofcode.com/2018/day/6

This problem is very suited to an array language like APL. We create a 3D matrix where each major cell is a 2D matrix holding the manhattan distances from each point to the input coordinate. We can then process this along the z-axis to pick the layer index of the smallest distance. That gives us a "Nearest" map. What remains is to remove any "infinites", defined as the numbers which occur along any edge of the "Nearest" map, and finally find the highest frequency. 

The Python [solution](https://github.com/xpqz/aoc-18/blob/master/day6.py) is considerably longer and slower.

In [128]:
⎕io←0
DAY6←⍎¨lines'data/2018/06.txt'

In [129]:
(XMAX YMAX)←⊃↓⍉↑⌈/DAY6

For each of the 50 points, make an array where each value is the MHD from the point.

In [130]:
MHD←{+/|⍵-⍺}
Dist←{YMAX XMAX⍴⍵∘MHD¨,⍳YMAX XMAX}

Create the nearest distance map. A small hoop to jump through: points that are equidistant to several coordinates should be removed. We set those to ¯1.

In [131]:
Nearest←{v←⍋⍵⋄⍵[v[0]]=⍵[v[1]]:¯1⋄⊃v}¨↓[0]↑Dist¨⌽¨DAY6 ⍝ Flip each input pair so we have (y x)

Remove any items touching the boundaries of the array, as they're 'infinite'.

In [132]:
⊢Part1←1⊃0⌷1 sortbycol{⍺(≢⍵)}⌸(∊Nearest)~∪Nearest[⍸(⌽∨⊖)0∊¨⍳⍴Nearest]
assert 3989=Part1

3989

For part 2, we instead seek the set of points that each have a combined MHD to all coords < 10,000.

In [133]:
⊢Part2←+/{10000>+/⍵∘MHD¨DAY6}¨,⍳YMAX XMAX
assert 49715=Part2

49715

### Day 7: The Sum of Its Parts
https://adventofcode.com/2018/day/7

We have a graph which we need to traverse in a specific child-order - sorting the queue each iteration is a somewhat inefficient approach, but as the number of nodes is tiny it doesn't really matter.

In [123]:
DAY7←'^.+?([A-Z]).*([A-Z]).*$'⎕r'\1\2'⊢lines'data/2018/07.txt'

In [124]:
(KEY PRE)←↓⍉↑(↓⊃¨,∘⊂⌸⊢/¨)↓⌽↑DAY7 ⍝ Make the graph - splitting out nodes and their ascendants

In [125]:
Day7p1←{0=≢⍵:⍺⋄(⍺,p)∇(⍺,p)~⍨sorted ⍵∪{KEY/⍨⍵∘{∧/⍵∊⍺}¨PRE} ⍺,p←⊃⍵}

In [126]:
⊢Part1←⍬ Day7p1 sorted KEY~⍨∪∊DAY7
assert 'BHMOTUFLCPQKWINZVRXAJDSYEG'≡Part1

┌→─────────────────────────┐
│BHMOTUFLCPQKWINZVRXAJDSYEG│
└──────────────────────────┘

Part 2 -- process items concurrently in a worker pool of 5. Each job now takes 60 + a number corresponding to the item's position in the alphabet (A=60+1, B=60+2 etc). Replaces 200+ lines of my Python [solution](https://github.com/xpqz/aoc-18/blob/master/day7.py).

In [127]:
Workers←{0=slots←5-≢⍺:⍺⋄⍺,{⍵,⍨61+⎕A⍳⍵}¨(slots⌊≢⍵)↑⍵}

In [130]:
]dinput
Day7p2←{
    wrk←⍬ ⋄ sec←0
    ⍬ {
        (0=≢⍵)∧0=≢wrk:sec
        (remains jobs)←↓⍉↑wrk⊢←¯1∘+@0¨wrk Workers ⍵ ⍝ Fill any available slots and 'tick'
        wrk⊢←wrk/⍨0≠remains                         ⍝ Reap any dead worker processes
        sec+←1
        (⍺,sorted ready)∇(⍺,jobs)~⍨⍵∪{KEY/⍨⍵∘{∧/⍵∊⍺}¨PRE}⍺,ready←jobs/⍨0=remains
    } ⍵
}

In [131]:
⊢Part2←Day7p2 sorted KEY~⍨∪∊DAY7
assert 877=Part2

877

### Day 8: Memory Maneuver
https://adventofcode.com/2018/day/8

Release the recursions!

In [99]:
DAY8←⍎line'data/2018/08.txt'

In [100]:
get←{v←DAY8[CUR+⍳⍵]⋄CUR+←⍵⋄v} ⍝ Poor-man's generator

In [101]:
Day8p1←{(chc mtc)←get 2⋄ch←⍬ {⍵=0:⍺⋄(⍺,Day8p1⍬)∇⍵-1} chc⋄⊂ch (get mtc)}

In [102]:
CUR←0
⊢Part1←+/∊TREE←Day8p1⍬
assert 44838=Part1

44838

In [103]:
Day8p2←{(ch mt)←⍵⋄0=≢ch:+/mt⋄ind←¯1+mt~0⋄+/∇¨ch[(ind<≢ch)/ind]}

In [104]:
⊢Part2←Day8p2 ⊃TREE
assert 22198=Part2

22198

### Day 9: Marble Mania
https://adventofcode.com/2018/day/9

This will tax systems that are O(N) for vector insert or delete. We need a structure that's O(1) for both insert and delete for part 2 at least, here in the shape of a bi-directional linked list. Ugly, but does the job.

In [60]:
ELVES←430⍴0
CIRCLE←{⍵⊣⍵.(v l r pos start)←(,0) (,0) (,0) 0 0}⎕NS'' 

In [42]:
val←{⍵.pos⊃⍵.v}
rev7←{l←⍵ ⋄ {⍵=¯1:l ⋄ l.pos←l.pos⊃l.l ⋄ ∇ ⍵-1}7}
fwd←{⍵.pos←⍵.pos⊃⍵.r⋄⍵}
twenty3←{v←val rev7 CIRCLE⋄_←fwd del CIRCLE⋄ELVES[⍺]+←⍵+v⋄⍬}

In [43]:
]dinput
add←{
    ⍺.v,←⍵
    ⍺.l,←⍺.pos
    ⍺.r,←right←⍺.pos⊃⍺.r
    ⍺.pos←⍺.l[right]←⍺.r[⍺.pos]←¯1+≢⍺.v
    ⍺
}

In [44]:
]dinput
del←{
    left←⍵.pos⊃⍵.l
    ⍵.r[left]←right←⍵.pos⊃⍵.r
    ⍵.l[right]←left
    ⍵.pos=⍵.start:⊢⍵.pos←⍵.start←right
    ⍵.pos←right
    ⍵
}

In [61]:
]dinput
Day9←{
    max←⍵
    0 {
        max<⍵:⌈/ELVES
        0=23|⍵:((≢ELVES)|⍺+1)∇(⍵+1)⊣⍺ twenty3 ⍵
        _←fwd CIRCLE add ⍵
        ((≢ELVES)|⍺+1)∇⍵+1
    } 1
}

In [62]:
⊢Part1←Day9 71588
assert 422748=Part1

422748

For part 2, we're asked to increase the marble count by two orders of magnitude.

In [58]:
ELVES←430⍴0
CIRCLE←{⍵⊣⍵.(v l r pos start)←(,0) (,0) (,0) 0 0}⎕NS''

In [59]:
⎕PP←11                     ⍝ Up the precision limit for printing floats
⊢Part2←Day9 7158800        ⍝ Takes ~15s to run
assert 3412522480=Part2

3412522480

### Day 10: The Stars Align
https://adventofcode.com/2018/day/10

Back on solid array footing for day 10 - a task ideally suited for APL. Repeatedly add the velocity vector until the bounding box of the points no longer decreases.

In [74]:
⎕IO←0
DAY10←↑⍎¨¨'(-?\d+)'⎕s'\1'¨lines'data/2018/10.txt'

In [75]:
POS←,/DAY10[;0 1]⋄DELTA←,/DAY10[;2 3]

In [76]:
Bbox←{+/|-⌿2 2⍴(⌊⌿,⌈⌿)↑⍵} ⍝ Width + height of bounding box

In [77]:
MSG←DELTA-⍨DELTA∘{⍵+⍺}⍣{(Bbox ⍺)>Bbox ⍵}⊢POS ⍝ Add velocity until bounding box no longer decreases.

In [78]:
(W H)←|-⌿2 2⍴(⌊⌿,⌈⌿)↑MSG ⍝ Width and height of bounding box

In [79]:
⍉(-(W+1))↑⍉(-(H+1))↑'*'@(⊖¨MSG)⊢143 203⍴' ' ⍝ Prune our display: FPRBRRZA

┌→─────────────────────────────────────────────────────────────┐
↓****** ***** ***** ***** ***** ***** ****** ** │
│* * * * * * * * * * * * * * │
│* * * * * * * * * * * * * *│
│* * * * * * * * * * * * * *│
│***** ***** ***** ***** ***** ***** * * *│
│* * * * * * * * * * * ******│
│* * * * * * * * * * * * *│
│* * * * * * * * * * * * *│
│* * * * * * * * * * * * *│
│* * * * ***** * * * * ****** * *│
└──────────────────────────────────────────────────────────────┘

Part 2: how many iterations was that?

In [80]:
I←0
_←DELTA∘{⍵+⍺}⍣{I+←1⋄(Bbox ⍺)>Bbox ⍵}⊢POS

In [81]:
⊢Part2←I-1
assert 10027=Part2

10027

### Day 11: Chronal Charge
https://adventofcode.com/2018/day/11

More APL home court advantage here.

In [142]:
⎕IO←1        ⍝ Note: 1-based indexing
DAY11←7803

In [143]:
P←{¯5+1⊃10 10 10⊤rack×DAY11+⍺×rack←⍵+10} ⍝ The power function, as defined by the question

We apply the power function P to a 300×300 grid and then use stencil to define all 3×3 neighbourhoods which we then sum up. Note that stencil used this way creates neighbourhoods that straddle the boundaries, so we need to subtract 1,1 from the result. To be completely general we should probably have excluded these boundary neighbourhoods.

In [144]:
G←300 300⍴P/↑,⍳300 300
SUM←{+/,⍵}⌺3 3⊢G

In [145]:
MAX←⌈/,SUM

In [146]:
⊢Part1←¯1+⊖⊃⍸MAX⍷SUM ⍝ Question specifically asks for x,y so we flip the coordinate pair
assert 20 51≡Part1   ⍝ Submittable result format: 20,51

┌→────┐
│20 51│
└~────┘

Part 2 asks us to find the neighbourhood of any size that gives the largest sum. There are possibly smart ways to achieve this, noting that we could reuse smaller neighbourhood sums when calculating larger ones. But let's start with the simple way, looking for some probable patterns.

Let's consider all regions up to 50×50 -- reasonably quick:

In [147]:
G∘{⌈/,{+/,⍵}⌺⍵ ⍵⊢⍺}¨3+⍳50

┌→───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│36 55 58 70 73 75 76 99 101 117 117 116 119 125 122 113 110 96 74 81 73 71 63 64 57 62 55 43 13 6 2 ¯6 ¯32 ¯52 ¯72 ¯79 ¯101 ¯108 ¯117 ¯117 ¯117 ¯117 ¯127 ¯127 ¯136 ¯136 ¯146 ¯146 ¯151 ¯151│
└~───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

We can see that the max sum (125) is at size 17, and then the values wobble downwards. So for part 2, we simply apply the same process as for part 1, but for a region size of 17×17, noting that we need to subtract 8,8 to skip the 17×17 neighbourhoods that aren't fully inside the boundary.

In [148]:
SUM←{+/,⍵}⌺17 17⊢G
MAX←⌈/,SUM
⊢Part2←¯8+⊖⊃⍸MAX⍷SUM  ⍝ Skip all neighbourhoods straddling a boundary
assert 230 272≡Part2  ⍝ Submittable result format: 230,272,17

┌→──────┐
│230 272│
└~──────┘

### Day 12: Subterranean Sustainability
https://adventofcode.com/2018/day/12

In [231]:
⎕IO←0
DAY12←'#'⎕R'1'¨'\.'⎕R'0'¨lines'data/2018/12.txt'

In [232]:
STATE←∊0 0,(⍎¨15↓0⊃DAY12),50⍴0 ⍝ Start state, as binary vector, padded a bit to the left and right
PATT←⍎¨¨5↑¨2↓DAY12             ⍝ Rule pattern bodies, as binary vectors
NEW←⍎¨¯1↑¨2↓DAY12              ⍝ Rule results as binary vector
RULES←NEW[⍋2⊥¨PATT]            ⍝ Sort rule results by the patterns converted from binary to decimal 0-31

The generational transform is now simply each overlapping binary 5-neighbourhood converted to decimal as indices into the RULES vector.

In [233]:
Next←{RULES[2⊥¨{⊂⍵}⌺5⊢⍵]}      ⍝ Transformation function

Part 1: run for 20 generations and sum up the pot-numbers for pots containing live plants. Note: not the number of live plants!

In [235]:
⊢Part1←+/¯2+⍸Next⍣20⊢STATE     ⍝ Remember to remove the left-side padding (¯2)
assert 3421=Part1

3421

Part 2: FIDDY BEEEEEEELLLION iterations! Obviously need to look at some data dependency trick here. Working hypothesis: check for emergent stable pattern. 

The two simplest potential options are:

1. Constant -- sum converges to constant value
2. Constant change -- sum _change_ converges to constant value

We can confirm that that the second case is what we have - let's look at the sums over time:

In [253]:
P2STATE←∊0 0,(⍎¨15↓0⊃DAY12),200⍴0         ⍝ Give ourselves a bit more growth room to the right
GENS←,+/¯2+⍸STATE
_←{v←Next ⍵⋄GENS,←+/¯2+⍸v⋄v}⍣150⊢P2STATE  ⍝ 150 generations

In [254]:
DIFFS←2-/GENS ⍝ Subtract current from previous; increases will show as negative

If we inspect how the generational differences change over the generations, we can see that after the 97th generation, every subsequent step adds 51 to the value:

In [255]:
97↓DIFFS

┌→──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│¯48 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51 ¯51│
└~──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [264]:
⎕PP←20
⊢Part2←(98⊃GENS)+51×50000000000-98
assert 2550000001195=Part2

2550000001195

### Day 13: Mine Cart Madness
https://adventofcode.com/2018/day/13

In [317]:
DAY13←↑lines'data/2018/13.txt'

Pick out the carts. 

We represent the compass bearings as 0, 1, 2, 3 for East, South, West and North respectively. The "turn state" for each cart defines its behaviour at intersections: left (0), straight (1), right (2).

In [318]:
⍝ row, col, heading, turn state
WEST ←(W←⍸'<'⍷DAY13),¨⊂0 0 1 2
SOUTH←(S←⍸'v'⍷DAY13),¨⊂1 0 1 2
EAST ←(E←⍸'>'⍷DAY13),¨⊂2 0 1 2
NORTH←(N←⍸'^'⍷DAY13),¨⊂3 0 1 2

"Fill in" the tiles the carts are sitting on. No cart is starting on an intersection or turn.

In [319]:
DAY13[W,E]←'-'
DAY13[N,S]←'|'

Part 1: what is the location of the first collision?
Part 2: each time we detect a collision, remove the two "wrecks" and carry on - what is the location of the last remaining cart at the moment of the final collision? 

As most of the bits are the same for part 1 and part 2, we can solve both at once.

In [339]:
]dinput
Day13←{
    graph←⍺
    loc←1@(↓(↑⍵)[;0 1])⊢150 150⍴0 ⍝ State: location of carts.
    Next←{
        dir←2⊃⍵ ⍝ Current bearing
        Rot←{∊(4 3⍴1 0 3 2 1 0 3 2 1 0 3 2)[0⊃⍵;1⊃⍵],1⊖1↓⍵} ⍝ Return new bearing and turn state
        
        Step←{
            ⍝ Move one step. Takes a left arg as a vector: dy dx turn1 turn2, and right arg a cart.
            ⍝ Move as defined by unit vector (dy, dx). Turn1 and turn2 are 
            ⍝ the new bearings when landing on a "/" or "\" respectively.
            ⍝ Thus: 0 ¯1 1 3 means "go one step east. If landing on "/", turn 
            ⍝ south. If "\", turn north.
            (dy dx slash backslash)←⍺
            loc[0⊃⍵;1⊃⍵]-←1
            pos←(dy+0⊃⍵),dx+1⊃⍵          ⍝ Step in stated direction
            loc[0⊃pos;1⊃pos]+←1
            tile←pos⌷graph               ⍝ Check the tile we landed on:
            '/'=tile:∊pos,slash,3↓⍵        ⍝ Turn: /
            '\'=tile:∊pos,backslash,3↓⍵    ⍝ Turn: \
            '+'=tile:∊pos,Rot 2↓⍵          ⍝ Intersection: +
            ∊pos,2↓⍵                       ⍝ Straight on: - or |
        }
        (dir⊃(0 ¯1 1 3)(1 0 0 2)(0 1 3 1)(¯1 0 2 0)) Step ⍵
    }

    crashes←⍬ ⍝ Store crash locations; only need first one for Part 1.
    Crash←{
        pos←2↑⍺
        crashes,←⊂pos
        loc[0⊃pos;1⊃pos]←0
        ⍵/⍨~(⊂pos)≡¨2↑¨⍵
    }
    MoveAll←{ ⍝ ⍺ are carts that have moved, ⍵ carts yet to move
        ⍺←⍬
        0=≢⍵:sorted ⍺
        cart←Next 0⊃⍵
        pos←2↑cart
        ⍝ If we have a 2 in the loc matrix, this cart collided with someone.
        2∊loc:(pos Crash ⍺)∇pos Crash 1↓⍵
        (⍺,⊂cart)∇1↓⍵
    }
    (⊂⊖0⊃crashes),⊂⊖2↑⊃MoveAll⍣{1=≢⍺}⊢⍵ ⍝ Repeat until only one remaining cart
}

In [340]:
⊢Solution←DAY13 Day13 WEST,SOUTH,EAST,NORTH
assert (76 108)(2 84)≡Solution

┌→────────────────┐
│ ┌→─────┐ ┌→───┐ │
│ │76 108│ │2 84│ │
│ └~─────┘ └~───┘ │
└∊────────────────┘

### Day 14: Chocolate Charts
https://adventofcode.com/2018/day/14

In [373]:
DAY14←765071
Mix←{new←10 10⊤+/⍺[⍵]⋄0≠0⊃new:new⋄1⊃new}

In [374]:
]dinput
Day14←{
    acc←3 7
    ⍵ {
        (≢acc)≥⍺+10:⍺↓acc
        acc,←acc Mix ⍵
        ⍺∇(≢acc)|1+⍵+acc[⍵]
    } 0 1
}

In [375]:
⊢Part1←Day14 DAY14
assert 3 1 7 1 1 2 3 9 2 3≡Part1

┌→──────────────────┐
│3 1 7 1 1 2 3 9 2 3│
└~──────────────────┘

Part 2: how many recipes to the left of us once the given pattern appears? There is a small gotcha here -- we can't just check for the pattern at the end of the vector of recipes as we sometimes add one, and sometimes two new recipes -- we also need to check one step in from the end.

We can try the new "over" operator from Dyalog v18 here: `X-⍥≢Y` gives us the difference in lengths between two vectors, which is an improvement from the fugly `(≢X)-≢Y`.

In [376]:
]dinput
Day14p2←{
    acc←3 7
    patt←(10∘⊥⍣¯1)⍵                         ⍝ The pattern is a vector of the digits of the number
    {
        patt≡acc↑⍨-≢patt:acc-⍥≢patt         ⍝ Check from end
        patt≡¯1↓acc↑⍨-1+≢patt:¯1+acc-⍥≢patt ⍝ Also check one step from end 
        acc,←acc Mix ⍵
        ∇(≢acc)|1+⍵+acc[⍵]
    } 0 1
}

In [377]:
⊢Part2←Day14p2 DAY14   ⍝ Takes ~20s to run
assert 20353748=Part2

20353748

### Day 15: Beverage Bandits
https://adventofcode.com/2018/day/15

This was a very difficult problem to get right -- there are many subtle rules that govern how the pieces move.

Note: when a unit moves within range of an enemy, it attacks as part of the same round:

A 'turn' resolves to:

1. If I am adjacent to one or more enemies, attack the first one in 'read order' (y-then-x)
2. Otherwise, move one step along the shortest path towards an enemy, tie-breaking on 'read order'
3. If, after a move, I am adjacent to one or more enemies, attack the first one in 'read order' (y-then-x)

For the moving of creatures we do a breadth-first search until the first attackable enemy is encountered, and then use the length of that path as a cut-off, thus chopping down the search space dramatically. The question suggests a different approach: identify all attacking squares first, find the shortest path to each of these. That would enable us to use A* with a manhattan distance heuristic. 

In [18]:
DAY15←↑lines'data/2018/15.txt'

In [49]:
]dinput
Day15←{
    (g e cave)←('G'⍷⍵)('E'⍷⍵)('.'⍷⍵)
    elfCount←+/∊e
    cave[(⍸g),⍸e]←1                   ⍝ Fill in tiles under feet of creatures
    g[⍸g]←200 ⋄ e[⍸e]←¯200            ⍝ Initial hitpoints: 200
    creatures←g+e
    N4←(⍸cave)∘{⍵≡⍬:⍬⋄⍺∩⍵(+,-)(0 1)(1 0)}   ⍝ Valid 4-connected neighbouring tiles
    edp←⍺ ⍝ elf damage points; for part 2

    Attack←{ ⍝ Reduce creature with least hitpoints by 3, towards 0
        0=≢⍵:⍬
        (_ y x)←⍵⊃⍨⊃⍋|⍵     ⍝ Absolute value to ensure correct sort order
        hp←creatures[⊂y x]  ⍝ Find original signed value
        ⊢creatures[⊂y x]⊢←(1=×hp)⌷(⌊/0,hp+3),⌈/0,hp-edp
    }

    Select←{ ⍝ Given a set of paths to nearby enemies, pick the next square.
        len←≢¨⍵
        best←len[⊃⍋len]             ⍝ Shortest length
        cand←(best≥len)/⍵           ⍝ We may have multiple
        1⌷⊃cand[⊃⍋{⊃¯1↓¯2↑⍵}¨cand]  ⍝ Last-but-one step (attack squares), read-order
    }

    Move←{ ⍝ Move creature at ⍵ one step
        (us them)←⍺

        0≠≢them∩N4 ⍵:⍵ ⍝ In attacking position already; remain

        max←⌊/⍬⋄found←⍬
        StorePaths←{found,←⍵⋄max⊢←⌊/≢¨found⋄⍬}

        BFS←{ ⍝ Search for shortest paths to enemy: basic breadth-first search
            0=≢⍵:⍬
            path←0⊃⍵ ⋄ queue←1↓⍵
            max<≢path:⍺∇queue
            neighbours←(⍺~⍨N4 ¯1↑path)~us
            attacking←neighbours∩them
            0≠≢attacking:(⍺,neighbours~them)∇queue⊣StorePaths path(↓,⍤1 0)attacking
            (⍺,neighbours)∇queue,path(↓,⍤1 0)neighbours
        }

        _←(,⍵) BFS ,⊂⍵
        0=≢found:⍬                 ⍝ No moves found
        next←Select found
        creatures[next]←creatures[⍵] ⋄ creatures[⍵]←0
        next
    }

    Round←{ ⍝ Process all creatures in 'read order': move + attack
        ⍺←⍬
        0=≢⍵:,⍸0≠creatures
        pos←0⌷⍵⋄tail←1↓⍵
        pos∊⍺:⍺∇tail  ⍝ Already seen this one this round; skip
        
        me←×creatures[pos]
        enemies←,⍸(¯1×me)=×creatures
        friends←,⍸me=×creatures

        newpos←(friends enemies)Move pos ⍝ Try move; stay still if attack pos
        asquares←enemies∩N4 newpos
        _←Attack creatures[asquares],¨asquares ⍝ Keep attacking

        (⍺,newpos)∇tail
    }

    count←0
    deadElf←0
    Winner←{ ⍝ Winner declared if no creatures of either sign
        count+←1
        types←∊×creatures
        elves←¯1=types
        PART2∧elfCount>+/elves:1⊣deadElf⊢←1  ⍝ Part 2: terminate on first dead elf
        (0=+/elves)∨0=+/1=×types
    }

    _←Round⍣Winner ⊢ ,⍸0≠creatures
    count deadElf (creatures)
}

In [51]:
PART2←0
(count deadElf creatures)←3 Day15 DAY15
⊢Part1←(count-1)×+/|∊creatures
assert 243390=Part1

243390

Part 2: find the smallest damage an elf needs to inflict in order such that the elves win the battle without any casualties.

In [52]:
PART2←1
⊢Part2←{(count deadElf creatures)←⍵ Day15 DAY15⋄deadElf=0:(count-1)×+/|∊creatures⋄∇⍵+1}4
assert 59886=Part2

59886

Ok, that was annoyingly fiddly! In retrospect, maybe we'd been better keeping the creatures in separate matrices instead of disambiguating by sign.

### Day 16: Chronal Classification
https://adventofcode.com/2018/day/16

We split the input file into the two obvious sections manually.

In [205]:
DAY16p1←↓774 3⍴⍎¨¨({0≠≢⍵}¨d)/d←'(\d+)'⎕S'\1'¨lines'data/2018/input16-1.data'
DAY16p2←⍎¨lines'data/2018/input16-2.data'

First, let's implement the various instructions. They all follow a small set of patterns - register operator register, or register operator value etc. We can exploit this with a couple of custom operators.

In [206]:
bin←{(32⍴2)⊤⍵}
rr←{((⍵[1]⌷⍺) ⍺⍺ (⍵[2]⌷⍺))@(3⌷⍵)⊢⍺}           ⍝ reg-reg
ri←{((⍵[1]⌷⍺) ⍺⍺ (⍵[2]))@(3⌷⍵)⊢⍺}             ⍝ reg-val
ir←{((⍵[1]) ⍺⍺ (⍵[2]⌷⍺))@(3⌷⍵)⊢⍺}             ⍝ val-reg
brr←{(2⊥(bin ⍵[1]⌷⍺) ⍺⍺ bin ⍵[2]⌷⍺)@(3⌷⍵)⊢⍺}  ⍝ reg-reg (binary)
bri←{(2⊥(bin ⍵[1]⌷⍺) ⍺⍺ bin ⍵[2])@(3⌷⍵)⊢⍺}    ⍝ reg-val (binary)
addr←+rr ⋄ addi←+ri
mulr←×rr ⋄ muli←×ri
banr←∧brr ⋄ bani←∧bri
borr←∨brr ⋄ bori←∨bri
setr←{(⍵[1]⌷⍺)@(3⌷⍵)⊢⍺} ⋄ seti←{⍵[1]@(3⌷⍵)⊢⍺}
gtir←>ir ⋄ gtri←>ri ⋄ gtrr←>rr
eqir←=ir ⋄ eqri←=ri ⋄ eqrr←=rr

In [207]:
OPS←'addr' 'addi' 'mulr' 'muli' 'banr' 'bani' 'borr' 'bori' 'setr' 'seti' 'gtir' 'gtri' 'gtrr' 'eqir' 'eqri' 'eqrr'

In [208]:
Validate←{(before instr after)←⍵⋄after≡before (⍎⍺) instr}

In [209]:
MATCH←OPS∘.Validate DAY16p1 ⍝ ♥ APL

In [210]:
⊢Part1←+/2>⍨+⌿MATCH
assert 517=Part1

517

For part 2 we need to resolve all the opcodes' names to numbers, and then execute the program, and return the final value left in register 0.

In [211]:
]dinput
Opcodes←{
    0=+/∊MATCH:⊢/↑sorted ⍵
    single←⊃⍸1=+⌿MATCH
    op←⊃1⊃⊃DAY16p1[single]
    row←⊃⍸MATCH[;single]
    MATCH[row;]←0
    ∇⍵,⊂op (row⊃OPS)
}

In [212]:
OPCODES←Opcodes ⍬

In [213]:
Run←{⍺←0 0 0 0⋄0=≢⍵:0⊃⍺⋄opc←⊃instr←0⊃⍵⋄(⍺ (⍎opc⊃OPCODES) instr)∇1↓⍵}

In [214]:
⊢Part2←Run DAY16p2
assert 667=Part2

667

### Day 17: Reservoir Research
https://adventofcode.com/2018/day/17

Another colossal cave ⍨

This problem has a simple and obvious recursive formulation, but sadly not tail-recursive. So let's use an explicit operations queue instead.

In [3]:
'iotag'⎕CY'dfns'
DAY17←lines'data/2018/17.txt'

In [4]:
]dinput
Parse←{
    col←⊃'x=(\d+), y=(\d+)\.\.(\d+)'⎕s'\1 \2 \3'⊢⍵
    0≠≢col:((1⊃⍎col)iotag 2⊃⍎col),¨0⊃⍎col
    row←⍎⊃'y=(\d+), x=(\d+)\.\.(\d+)'⎕s'\1 \2 \3'⊢⍵
    (0⊃row),¨(1⊃row)iotag 2⊃row
}

In [5]:
CLAY←⊃,/Parse¨DAY17

In [21]:
CAVE←'#'@CLAY⊢(1+⊃⌈/CLAY)⍴'.'
YMAX←¯1+⊃⍴CAVE

In [7]:
Open←{3::0⋄CAVE[⊂⍵]∊'.|'}
Closed←~Open
Boundary←{(y x dx)←⍵⋄{(Closed y,⍵+dx)∨Open ⍵,⍨y+1:⍵⋄∇⍵+dx}x}

In [8]:
]dinput
Down←{
    (y x)←⍵
    newY←YMAX⌊¯1+y+⌊/(y↓CAVE[;x])⍳'#~'
    CAVE[y+⍳newY-y-1;x]←'|'
    newY≠YMAX:⊂'S'newY x
    ⍬
}

In [9]:
]dinput
Spread←{
    (y x)←⍵
    left←Boundary y x ¯1
    right←Boundary y x 1
    stretch←left+⍳right-left-1
    CAVE[y;stretch]←'|'
    opens←(Open right,⍨y+1)(Open left,⍨y+1)((Closed left,⍨y+1)∧Closed right,⍨y+1)
    ops←opens/('D' y right)('D' y left)('S' (y-1) x)
    2⊃opens:ops⊣CAVE[y;stretch]←'~'
    ops
}

In [22]:
]dinput
Trickle←{ ⍝ ⍺ keeps track of visited nodes. ⍵ is the pending queue
    0=≢⍵:(+/'|'=∊CAVE) (+/'~'=∊CAVE)
    op←⊃0⌷⍵ ⋄ tail←1↓⍵
    (⊂op)∊⍺:⍺∇tail
    'D'=0⊃op:(⍺,⊂op)∇tail,Down 1↓op
    (⍺,⊂op)∇tail,Spread 1↓op
}

In [23]:
(Wet Settled)←⍬ Trickle ,⊂'D' 0 500

In [25]:
⊢Part1←Wet+Settled
assert 39370=Part1

39370

In [26]:
⊢Settled
assert 33061=Settled

33061

### Day 18: Settlers of The North Pole
https://adventofcode.com/2018/day/18

Conway's game of Lumber. This kind of problem is APL home turf.

Note: when reading in the data we rely on the `∪⍳⊢` trick to convert characters to their unique index of first occurrence. This means that the map will differ with the data set. In order for the below solution to work for a different data set, we'd need to ensure that

* 0 = ground (.)
* 1 = tree (|)
* 2 = lumber (#)

Tweaking the `Display` dfn below to do this is left as an exercise for the interested reader.

In [37]:
DAY18←50 50⍴(∪⍳⊢)∊lines'data/2018/18.txt' ⍝ 50×50: 0 = ground (.), 1 = tree (|), 2 = lumber (#)

In [64]:
Display←{'#'@(2∘=)⊢'|'@(1∘=)⊢'.'@(0∘=) ⍵} ⍝ Debugging dfn left for posterity/sanity

In [78]:
]dinput
TNG←{
    i←⍵[⊂1 1]
    frq←+/⍉↑(∊⍵)=⊂0 1 2 ⍝ Can't easily use ⌸ -- need missing values, too
    (i=0)∧frq[1]≥3:1
    (i=1)∧frq[2]≥3:2
    i≠2:i 
    (i=2)∧(frq[2]≥2)∧(frq[1]≥1):2 ⍝ Note: frq[2]≥2 -- disregard self
    0
}

In [79]:
FINAL←{TNG ⍵}⌺3 3⍣10⊢DAY18

In [80]:
TREES←+/1=∊FINAL ⋄ LUMBER←+/2=∊FINAL

In [81]:
⊢Part1←TREES×LUMBER
assert 589931=Part1

589931

So for part 2, the customary AoC trap-door: now for beeeeellion iterations, which is code for that there is a data dependency or recurring pattern to exploit.

For GoL-like cellular automata, especially on a fixed-size board, we know that we frequently encounter either a stable state (no changes), or a loop (a recurring pattern). We could be scientific about it, but by letting this run and visualising the state a prominent pattern quickly starts recurring. 

The state at 1×10^9 generations should be the same as that at 1000 generations.

In [84]:
FINAL←{TNG ⍵}⌺3 3⍣1000⊢DAY18

In [85]:
TREES←+/1=∊FINAL ⋄ LUMBER←+/2=∊FINAL
⊢Part2←TREES×LUMBER
assert 222332=Part2

222332

### Day 19: Go With The Flow
https://adventofcode.com/2018/day/19

More asmbunny VMing. We can repurpose bits of Day 16 above.

In [86]:
rr←{((⍵[1]⌷⍺) ⍺⍺ (⍵[2]⌷⍺))@(3⌷⍵)⊢⍺}           ⍝ reg-reg
ri←{((⍵[1]⌷⍺) ⍺⍺ (⍵[2]))@(3⌷⍵)⊢⍺}             ⍝ reg-val
addr←+rr ⋄ addi←+ri
mulr←×rr ⋄ muli←×ri
setr←{(⍵[1]⌷⍺)@(3⌷⍵)⊢⍺} ⋄ seti←{⍵[1]@(3⌷⍵)⊢⍺}
gtrr←>rr
eqrr←=rr

In [88]:
DAY19←lines'data/2018/19.txt'

In [104]:
IPREG←⍎4↓0⊃DAY19

In [114]:
CODE←{(⊂4↑⍵),⍎4↓⍵}¨1↓DAY19

In [118]:
]dinput
Run←{ ⍝ ⍺ is code, ⍵ is regs
    ip←IPREG⊃⍵
    ip≥≢⍺:0⊃⍵
    opc←⊃instr←ip⊃⍺
    regs←⍵ (⍎opc) instr
    regs[IPREG]+←1
    ⍺∇regs
}

In [120]:
⊢Part1←CODE Run 0 0 0 0 0 0
assert Part1=2640

2640

Part 2 requires a different approach. Starting with `reg[0]←1` is a very different proposition. There are two nested loops, the key one the section at ip=3-11. In all it basically looks like this:

    reg←0 1 10551432 1 1 3 ⍝ state when we enter loop
     
    :While reg[3]≤reg[2]
        reg[1]←1
        :While reg[1]≤reg[2]
            :If (reg[3]×reg[1])=reg[2]
                reg[0]+←reg[3]
            :EndIf
            reg[1]←1
        :EndWhile
        reg[3]+←1
    :EndWhile
   
so that's two nested loops to 10,551,432 -- too much to run on this machine.

However, we can optimise away the inner loop completely, as shown below, as it only modifies reg[0] whenever reg[3] divides reg[2].

In [132]:
]dinput
Day19p2←{
    ⍵[3]>⍵[2]:⍵[0]
    0=⍵[3]|⍵[2]:∇(⍵[0]+⍵[3]),⍵[1],⍵[2],⍵[3]+1
    ∇1+@3⊢⍵
}

In [133]:
⊢Part2←Day19p2 0 1 10551432 1
assert 27024480=Part2

27024480

### Day 20: A Regular Map
https://adventofcode.com/2018/day/20

Traverse a maze and find the longest shortest path between the start point and any other point. For part 2, count the number of points reachable in at least 1000 steps. Depth-first search. Message for my future self: is there a better array programming solution to be had here?

In [146]:
DAY20←1↓¯1↓line'data/2018/20.txt'

In [147]:
]dinput
Day20←{
    (y x py px)←0
    pattern←⍵
    loc←1500⌶,⊂y x
    dist←,0
    Update←{ ⍝ Bleurgh. Half the kindom for a hash table!
        (⊂y x)∊loc:dist[loc⍳⊂y x]←dist[loc⍳⊂y x]⌊1+dist[loc⍳⊂py px]
        dist,←nd⊣loc,←(⊂y x)⊣nd←1+dist[loc⍳⊂py px]
        ⍬
    }
    Move←{('NESW'⍳⍵)⊃(¯1 0)(0 1)(1 0)(0 ¯1)}
    ⍬ { ⍝ Depth-first traverse
        0=≢⍵:dist
        (py px)⊢←y x
        ch←1↑⍵
        '('=ch:(⍺,⊂y x)∇1↓⍵            ⍝ Push
        ')'=ch:(¯1↓⍺)∇1↓⍵⊣(y x)⊢←⊃¯1↑⍺ ⍝ Pop
        '|'=ch:⍺∇1↓⍵⊣(y x)⊢←⊃¯1↑⍺      ⍝ Branch
        (y x)+←Move ch                 ⍝ Move
        ⍺∇1↓⍵⊣Update⍬                  ⍝ Store shortest path to this loc
    } pattern
}

In [148]:
dist←Day20 DAY20
⊢Part1←⌈/dist
assert Part1=3930

3930

In [149]:
⊢Part2←+/dist≥1000
assert Part2=8240

8240

### Day 21: Chronal Conversion
https://adventofcode.com/2018/day/21

Ok, the whole asmbunny thing is becoming rather tiring now.

Again, some hand-disassembly is required. Part 2 is not naively brute-forceable in APL. [It is a ~20s wait in C++](https://gist.github.com/xpqz/0b7a4e583f6f0080a50f0904bac900d4), and [an hour in Python](https://github.com/xpqz/aoc-18/blob/master/day21.py).

Part1: looking at the data, we see that line 28 is the
exit condition we need to trigger. Run the program until
the first time line 28 is hit, and the answer is the value
in register[2]

    ip--inst-------A--------B-C
    ---------------------------
    0:  seti     123        0 2		
    1:  bani       2      456 2
    2:  eqri       2       72 2 COND reg[2] == 72
    3:  addr       2        1 1
    4:  seti       0        0 1 GOTO 1
    5:  seti       0        3 2		
    6:  bori       2    65536 5 REG 5 = REG 2 | 65536
    7:  seti 4843319        1 2		
    8:  bani       5      255 4
    9:  addr       2        4 2
    10: bani       2 16777215 2     
    11: muli       2    65899 2		
    12: bani       2 16777215 2
    13: gtir     256        5 4 COND 256 > reg[5]
    14: addr       4        1 1
    15: addi       1        1 1
    16: seti      27        4 1 GOTO 28
    17: seti       0        7 4		
    18: addi       4        1 3
    19: muli       3      256 3
    20: gtrr       3        5 3 COND reg[3] > reg[5]
    21: addr       3        1 1
    22: addi       1        1 1
    23: seti      25        0 1 GOTO 26
    24: addi       4        1 4
    25: seti      17        0 1 GOTO 18
    26: setr       4        1 5 REG 5 = REG 4 
    27: seti       7        3 1 GOTO 8
    28: eqrr       2        0 4 COND reg[2] == reg[0] ⍝ Only place reg[0] is consulted
    29: addr       4        1 1
    30: seti       5        3 1 GOTO 6

In [153]:
DAY21←lines'data/2018/21.txt'
IPREG←⍎4↓0⊃DAY21
CODE←{(⊂4↑⍵),⍎4↓⍵}¨1↓DAY21

In [157]:
bin←{(32⍴2)⊤⍵}
rr←{((⍵[1]⌷⍺) ⍺⍺ (⍵[2]⌷⍺))@(3⌷⍵)⊢⍺}           ⍝ reg-reg
ri←{((⍵[1]⌷⍺) ⍺⍺ (⍵[2]))@(3⌷⍵)⊢⍺}             ⍝ reg-val
ir←{((⍵[1]) ⍺⍺ (⍵[2]⌷⍺))@(3⌷⍵)⊢⍺}             ⍝ val-reg
brr←{(2⊥(bin ⍵[1]⌷⍺) ⍺⍺ bin ⍵[2]⌷⍺)@(3⌷⍵)⊢⍺}  ⍝ reg-reg (binary)
bri←{(2⊥(bin ⍵[1]⌷⍺) ⍺⍺ bin ⍵[2])@(3⌷⍵)⊢⍺}    ⍝ reg-val (binary)
addr←+rr ⋄ addi←+ri
mulr←×rr ⋄ muli←×ri
banr←∧brr ⋄ bani←∧bri
borr←∨brr ⋄ bori←∨bri
setr←{(⍵[1]⌷⍺)@(3⌷⍵)⊢⍺} ⋄ seti←{⍵[1]@(3⌷⍵)⊢⍺}
gtir←>ir ⋄ gtri←>ri ⋄ gtrr←>rr
eqir←=ir ⋄ eqri←=ri ⋄ eqrr←=rr

In [158]:
]dinput
Run←{ ⍝ ⍺ is code, ⍵ is regs
    ip←IPREG⊃⍵
    ip≥≢⍺:⍵
    ip=28:⍵
    opc←⊃instr←ip⊃⍺
    regs←⍵ (⍎opc) instr
    regs[IPREG]+←1
    ⍺∇regs
}

In [160]:
⊢Part1←2⊃CODE Run 0 0 0 0 0 0
assert Part1=8797248

8797248

Part 2: the brute-force approach would be to keep recording the reg[2] values seen at ip 28 until we have a recurrence. Unfortunately, that takes a loooong time -- too long (trust me, I tried).

...so we're back to actually figuring out what the asmbunny code _does_. After some long and tedious pen & paper scribbles and lots of binary bit twiddling later, we arrive at something like the below for the value in reg[2] at ip=28. We could of course have used this for part 1, too, but we'll leave the above as a warning for the unwary.

On the plus side - fast af.

In [190]:
]dinput
Day21p2←{
    SEEN,←⍵
    a←dec ⍵ or 65536
    b←4843319
    b+←dec a and 255 ⋄ b←dec b and 16777215
    b×←65899 ⋄ b←dec b and 16777215
    b+←dec (dec 8 rs bin a) and 255 ⋄ b←dec b and 16777215
    b×←65899 ⋄ b←dec b and 16777215
    b+←dec (dec 16 rs bin a) and 255 ⋄ b←dec b and 16777215
    b×←65899
    dec b and 16777215
}

In [191]:
SEEN←⍬ ⋄ _←Day21p2⍣{⍺∊SEEN}⊢0

In [192]:
⊢Part2←⊃¯1↑SEEN       ⍝ Last element before the recurring
assert 3007673=Part2

3007673

### Day 22: Mode Maze
https://adventofcode.com/2018/day/22

This is clearly the year of colossal caves and asmbunnies...

In [242]:
DEPTH←10689
TARGET←722 11 ⍝ Y X

In [246]:
]dinput
GI←{
    ¯1≠⊃CAVE[⊂⍵]:⊃CAVE[⊂⍵]
    (0 0≡⍵)∨TARGET≡⍵:CAVE[⊂⍵]←0
    (y x)←⍵
    y=0:CAVE[⊂⍵]←x×16807
    x=0:CAVE[⊂⍵]←y×48271
    ⊢CAVE[⊂⍵]←(EL y,x-1)×EL x,⍨y-1
}

In [247]:
EL←{20183|DEPTH + GI ⍵}

In [248]:
CAVE←(1+TARGET)⍴¯1
_←GI¨,⍳ 1+TARGET

In [239]:
RISK←3|20183|DEPTH+CAVE ⍝ ♥ APL

In [240]:
⊢Part1←+/∊RISK
assert 8575=Part1

8575

Well, that was nice, wasn't it?

Part 2 becomes another graph path finding exercise, with a small twist: at each location we carry a bit of state which affects the neighbourhood function. This means that for each location we can move to any other location as defined by the RISK array and our state. So our neighbourhood function must include a "stay-put-change-state" mode, which has a cost. Other than that, should be a standard cheapest path Dijkstra effort, cut down to only keep track of path cost, not the path itself.

As I recall, we have one of those on tap since a previous year of this.... ah yes, day 13, 2016. 

A complication is that the graph/cave is essentially unbounded. We need some kind of heuristic cut-off which we can trial-and-error by just enlarging the RISK array a bit, and cutting off coordinates that drift too far away from the target.

In [241]:
]LINK.Create heapq src/heapq ⍝ All good graph algorithms need a priority queue

┌→──────────────────────────────────────────────────────────────┐
│Linked: #.heapq → /Users/stefan/work/dyalog/AoCDyalog/src/heapq│
└───────────────────────────────────────────────────────────────┘

TODO: rewrite this horror as a dfn..

In [254]:
]dinput
R←N4 arg;y;x;tool;squares;s;vals;square;regionTool;newTool;cost
(y x tool)←arg ⍝ tool → currently hefted tool
squares←(~+/¨¯1=s)/s←(⊂y x)+(+,-)(0 1)(1 0)
vals←⍬
:For square :In squares
    :If 0 0≢square≥⍴RISK
        :Continue
    :EndIf
    regionTool←RISK[⊂square]
    :For newTool :In ⍳3
        :If regionTool≠newTool
            cost←1
            :If tool ≠ newTool
                cost←8
            :EndIf
            vals,←⊂∊square newTool cost
        :EndIf
    :EndFor
:EndFor
R←vals

In [253]:
]dinput
Dijkstra←{
    ⍝ A spin on the classic Dijkstra best-first-search algorithm.
    ⍝ Only interested in the path cost, not the path itself.
    costKeys←1500⌶,⍵                                           
    costVals←,0
    Cost←{⍵∊costKeys:costVals[⊃costKeys⍳⍵] ⋄ ⌊/⍬} ⍝ Default cost is +infinity
    _←{
        (frontier queueItem)←##.heapq.Pop ⍵
        (totalCost current)←queueItem
        (⊃current)≡TARGET,1:⍬
        frontier {
            0=≢⍵:⍺
            (y x tool cost)←0⊃⍵
            newCost←cost+totalCost
            0 0≢y x≥2×TARGET:⍺∇1↓⍵       ⍝ Heuristic cut-off at 2× TARGET
            newCost≥Cost ⊂TARGET,1:⍺∇1↓⍵ ⍝ Too costly!
            newCost≥Cost ⊂y x tool:⍺∇1↓⍵ ⍝ Old pos, but at worse cost; bail
            queue←⍺ ##.heapq.Push ⊂newCost(⊂y x tool)
            (⊂y x tool)∊costKeys:queue∇(1↓⍵)⊣costVals[⊃costKeys⍳⊂y x tool]←newCost
            costKeys,←⊂y x tool ⋄ costVals,←newCost                            
            queue∇1↓⍵   
        } N4 ⊃current
    }⍣{##.heapq.Empty ⍺} ##.heapq.Push⊂0 ⍵                    
    ¯2+Cost ⊂TARGET,1 ⍝ Remove endpoint steps
}

In [249]:
CAVE←800 25⍴¯1

In [250]:
_←GI¨,⍳ 800 25

In [251]:
RISK←3|20183|DEPTH+CAVE

In [256]:
⊢Part2←Dijkstra ⊂0 0 1
assert 999=Part2

999

### Day 23: Experimental Emergency Teleportation
https://adventofcode.com/2018/day/23

This is a difficult problem, even if it starts easy.

In [278]:
DAY23←↑⍎¨¨'-?\d+'⎕s'&'¨lines'data/2018/23.txt'
RADII←⊢/DAY23
CENTRES←DAY23[;⍳3]

The strongest bot is that which has the largest radius.

In [279]:
STRONGEST←⊃⍒RADII

For part 1, we seek the total number of bots whose centres fall within the radius of the strongest bot, identified above.

In [281]:
MhD←{+/|⍺-⍵}

In [285]:
BOSS←STRONGEST⌷CENTRES

In [290]:
⊢Part1←+/(STRONGEST⊃RADII)≥BOSS∘MhD ¨↓CENTRES
assert 704=Part1

704

And so for part 2, we seek the point closest to the origin which is in reach of the largest number of bots. This is a hard problem, quite possibly the hardest of all AoC problems 2015-2018.

After failing to solve this a lot, it turns out that problem can be reduced to a projection from 3d to 1d. A couple of joyful mathematical deep-dives on [Numberphile](https://www.youtube.com/channel/UCoxcjq-8xIDTYp3uz647V5A) give some clues as to why this works as it does:

* [Balls and Cones](https://www.youtube.com/watch?v=lubGnk0UZt0)
* [Earthquakes, Circles and Spheres](https://www.youtube.com/watch?v=2vnqSwWAn34&list=PLt5AfwLFPxWI9eDSJREzp1wvOJsjt23H_)

The solution below was inspired by 

https://www.reddit.com/r/adventofcode/comments/a8s17l/2018_day_23_solutions/ecdqzdg/

We project each sphere onto a line so that we end up with a range -radius to radius centered around the point corresponding to the manhattan distance to the centre. We then create a vector, ordered by distance, of 1 for the start of a range and ¯1 for the end of a range, indicating if the bot contributes to the nesting or not. We then find the maximum sum along the vector.

In [345]:
overlaps←{⊃,/,⊆¨⍵}⍣≡{d←+/|⍵⋄((0⌈d-⍺),1) (¯1,⍨d+⍺+1)}/RADII,⍪↓CENTRES ⍝

In [346]:
distOrder←0(⊢⌷⍨∘⊂∘⍋⌷⍤1)↑overlaps
closest←¯1+⌈/+\⊢/0(⊢⌷⍨∘⊂∘⍋⌷⍤1)distOrder  ⍝ ♥ APL -- scan can be pure magic...

In [347]:
⊢Part2←⊃closest⌷distOrder
assert 111960222=Part2

111960222

### Day 24: Immune System Simulator 20XX
https://adventofcode.com/2018/day/24

It's starting to feel a lot like Christmas.

This was annoying, too.

In [10]:
'segs'⎕CY'dfns'
DAY24←lines'data/2018/24.txt'

In [11]:
]dinput
Parse←{
    team←⍺
    types←'fire' 'bludgeoning' 'slashing' 'cold' 'radiation'
    numbers←⍎¨'\d+'⎕s'&'⊢⍵
    attack←⊃types⍳'\s([^\s]+)\sdamage'⎕s'\1'⊢⍵
    immunities←types⍳', '∘segs⊃'immune to ([^;)]+)'⎕s'\1'⊢⍵
    weaknesses←types⍳', '∘segs⊃'weak to ([^;)]+)'⎕s'\1'⊢⍵
    {⍵⊣⍵.(team units hp damage initiative attack immune weak)←team,numbers,attack,(⊂1@immunities⊢5⍴0),⊂1@weaknesses⊢5⍴0}⎕NS'' 
}

In [21]:
Dead←{⍵.units≤0}
EffectivePower←{⍵.units×⍵.damage}
AttackOrder←{⍵[⍒↓⍉↑(EffectivePower ⍵)({⍵.initiative} ⍵)]}

In [13]:
]dinput
Damage←{ ⍝ ⍺ is us, ⍵ is them
   (Dead ⍺)∨Dead ⍵:0
   ⍵.immune[⍺.attack]:0
   ep←EffectivePower ⍺
   ⍵.weak[⍺.attack]:2×ep
   ep
}

In [14]:
]dinput
Attack←{ ⍝ ⍺ attacks ⍵
    ⍵≡⍬:0
    ⍵.immune[⍺.attack]:0
    damage←×/⍺.units,⍺.damage,1 2[⍵.weak[⍺.attack]]
    killed←⌊damage÷⍵.hp
    ⍵.units-←killed
    killed
}

In [15]:
]dinput
Select←{
    targets←⍵
    ⍬ {
        0=≢⍵:(targets,⊂⍬)[⍺] ⍝ Inject ⍬ where no target was found
        attacker←0⊃⍵
        damage←attacker∘.Damage targets
        order←⍒↓⍉↑(damage)(EffectivePower targets)({⍵.initiative} targets)
        pick←order[⍸<\(~order∊⍺)∧0≠damage[order]]
        ⍬≡pick:(⍺,≢targets)∇1↓⍵
        (⍺,pick)∇1↓⍵
    } ⍺
}

In [24]:
]dinput
Day24←{
    {
        (immune infect)←⍵
        aoim←AttackOrder immune
        aoinf←AttackOrder infect
        cohort←aoim,aoinf
        initiative←⍒{⍵.initiative} cohort
        ordered←cohort[initiative]
        targets←((aoim Select infect),aoinf Select immune)[initiative]
        _←Attack⌿↑(ordered)(targets)
        ⍵
    }⍣{(immune infect)←⍺ ⋄ (∧/Dead infect)∨∧/Dead immune}⊢⍵
}

In [25]:
IMMUNE←0 Parse¨DAY24[1+⍳10]
INFECT←1 Parse¨DAY24[13+⍳10]
_←Day24 IMMUNE INFECT
⊢Part1←+/{0⌈⍵.units} IMMUNE,INFECT
assert 18346=Part1

18346